In [1]:
#Scrape and download all links to the local file system due to http speed limit
import urllib.request
html_doc = ''
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_Greek_mythological_figures') as response:
    html_doc = response.read()

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')
all_links = soup.find_all('a')
print(len(all_links))

2090


In [3]:
#Download all of the relevant web pages from all the links
def download_pages():
    counter = 0
    for link in all_links:
        if(link.has_attr('href') and "/wiki/" in link['href']):
            if counter/len(all_links) % 10 == 0:
                print(str(counter/len(all_links) * 100) + '%')
            href = link['href']
            if '.jpg' or '.svg' or '.png' in href:
                continue
            urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')
    href = "/wiki/List_of_Greek_mythological_figures"
    urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')

In [4]:
soup.h3.span.text
soup.h3.next_sibling
sibling = soup.h3
counter = 0
h3_tag = soup.h3
h3_sibling_content = soup.h3.next_sibling.next_sibling

#Go back 1 sibling to be able to grab the first h3 tag into the list
clean_siblings = [s for s in h3_tag.previous_sibling.next_siblings if (str(s).strip() != "" and (s.name == "h3" or s.name == "div" or s.name == "table" or s.name=="ul"))]
clean_siblings = clean_siblings[:-7]
clean_siblings.remove(clean_siblings[5])

it = iter(clean_siblings)
heading_content_dict= {}
for sibling in it:
    heading_content_dict[sibling.text.replace("[edit]","").replace("/"," & ")] = next(it)
print(len(heading_content_dict))
#print(heading_content_dict)
print(heading_content_dict.keys())
        
    #print(next(it))

19
dict_keys(['Major gods and goddesses', 'Primordial deities', 'Titans and Titanesses', 'Gigantes and other "giants"', 'Personified concepts', 'Chthonic deities', 'Sea deities', 'Sky deities', 'Rustic deities', 'Agricultural deities', 'Health deities', 'Other deities', 'Deified mortals', 'Heroes', 'Notable women', 'Kings', 'Seers & oracles', 'Amazons', 'Inmates of Tartarus'])


In [42]:
import re

doc = heading_content_dict["Major gods and goddesses"]
soup_major = BeautifulSoup(str(doc), 'html.parser')
values = soup_major.find_all("td")
sub_pages = []
for td in values:
    if not td.a.img:
        print(td.a['href'])
        f = open(td.a['href'][1:] + ".html", 'r')
        sub_doc = f.read()
        soup_major_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_major_sub.find_all("tr")
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents"):
                print(tr.th.text + ":")
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                print(re.sub(r'(\[|\().*?(\]|\))','',st).split(",")) #Parse out citation square brackets
        print()
        
        
    

/wiki/Aphrodite
Children:
['Eros', ' Phobos', ' Deimos', ' Harmonia', ' Pothos', ' Anteros', ' Himeros', ' Hermaphroditus', ' Rhodos', ' Eryx', ' Peitho', ' Eunomia', ' The Graces', ' Priapus', ' Aeneas']
Parents:
['Uranus', 'Zeus', 'Dione']

/wiki/Apollo
Children:
['Asclepius', ' Troilus', ' Aristaeus', ' Orpheus']
Parents:
['Zeus', 'Leto']

/wiki/Ares
Children:
['Erotes ', ' Phobos', ' Deimos', ' Phlegyas', ' Harmonia', ' Enyalios', ' Thrax', ' Oenomaus', ' Amazons', 'Adrestia']
Parents:
['Zeus', 'Hera']

/wiki/Artemis
Parents:
['Zeus', 'Leto']

/wiki/Athena
Parents:
['Metis', 'Zeus']

/wiki/Demeter
Children:
['Persephone', ' Despoina', ' Arion', ' Plutus', ' Philomelus', ' Eubuleus', ' Chrysothemis', '', 'Amphitheus I']
Parents:
['Cronus', 'Rhea']

/wiki/Dionysus
Children:
['Priapus', ' Hymen', ' Thaos', ' Staphylus', ' Oenopion', ' Comus', ' Phthonus', ' the Graces']
Parents:
['Zeus', 'Persephone ', ' Zeus', 'Semele']

/wiki/Hades
Children:
['Macaria', ' Melinoe']
Parents:
['Cronus

In [77]:
doc = heading_content_dict["Primordial deities"]
soup_prime = BeautifulSoup(str(doc), 'html.parser')
values = soup_prime.find_all("tr")
links = []
for tr in values:
    if tr.contents[3].a:
        f = open(tr.contents[3].a['href'][1:] + '.html','r')
        print(len(f.read()))
        

65123
89091
90523
105967
85413
80658
135713
171071
69232
63369
160744
106538
56325
90797
220627
59677
76952
152303
119772
73932
74727


In [6]:
 from py2neo import Graph
